# Homework 2: Word Similarity

Student Name: Ruilin Liu

Student ID: 871076

## General info

<b>Due date</b>: Thursday, 18 Apr 2019 4pm

<b>Submission method</b>: see LMS

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -20% per day

<b>Marks</b>: 7% of mark for class (with 6% on correctness + 1% on quality and efficiency of your code)

<b>Materials</b>: See the main class LMS page for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib and Scikit-Learn. In particular, if you are not using a lab computer which already has it installed, we recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages; if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> You should use Python 3</b>. 

To familiarize yourself with NLTK, here is a free online book:  Steven Bird, Ewan Klein, and Edward Loper (2009). <a href=http://nltk.org/book>Natural Language Processing with Python</a>. O'Reilly Media Inc. You may also consult the <a href=https://www.nltk.org/api/nltk.html>NLTK API</a>.

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should edit the sections below where requested, but leave the rest of the code as is. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each section is worth is given in parenthesis after the instructions. 

You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Updates</b>: Any major changes to the assignment will be announced via LMS. Minor changes and clarifications will be announced in the forum on LMS, we recommend you check the forum regularly.

<b>Academic Misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.

## Overview

In this homework, you'll be quantifying the similarity between pairs of words of a dataset using different methods with the word co-occurrence in the Brown corpus and synset structure of WordNet. Firstly, you will preprocess the dataset to filter out the rare and ambiguous words. Secondly, you will calculate the similarity scores for pairs of words in the filtered dateset using Lin similarity, NPMI and LSA. Lastly, you will quantify how well these methods work by comparing to a human annotated gold-standard.

## 1. Preprocessing (2 marks)

<b>Instructions</b>: For this homework we will be comparing our methods against a popular dataset of word similarities called <a href="http://www.cs.technion.ac.il/~gabr/resources/data/wordsim353/">Similarity-353</a>. You need to first obtain this dataset, which is available on LMS. The file we will be using is called *set1.tab*. Make sure you save this in the same folder as the notebook.  Except for the header (which should be stripped out), the file is tab formated with the first two columns corresponding to two words, and the third column representing a human-annotated similarity between the two words. <b>You should ignore the subsequent columns</b>.

Here shows the first six lines of the file:

```
Word 1	Word 2	Human (mean)	1	2	3	4	5	6	7	8	9	10	11	12	13	
love	sex	6.77	9	6	8	8	7	8	8	4	7	2	6	7	8	
tiger	cat	7.35	9	7	8	7	8	9	8.5	5	6	9	7	5	7	
tiger	tiger	10.00	10	10	10	10	10	10	10	10	10	10	10	10	10	
book	paper	7.46	8	8	7	7	8	9	7	6	7	8	9	4	9	
computer	keyboard	7.62	8	7	9	9	8	8	7	7	6	8	10	3	9	
```
    
You should load this file into a Python dictionary (NOTE: in Python, tuples of strings, i.e. ("tiger","cat") can serve as the keys of a dictionary to map to their human-annotated similarity). This dataset contains many rare words: we need to filter this dataset in order for it to be better suited to the resources we will use in this assignment. So your first goal is to filter this dataset to generate a smaller test set where you will evaluate your word similarity methods.

The first filtering is based on document frequencies in the Brown corpus, in order to remove rare words. In this homework, we will be treating the paragraphs of the Brown corpus as our "documents". You can iterate over them by using the `paras` method of the corpus reader. You should remove tokens that are not alphabetic. Tokens should be lower-cased and lemmatized. Now calculate document frequencies for each word type, and use this to remove from your word similarity data any word pairs where at least one of the two words has a document frequency of less than 8 in this corpus.

For this part, you should store all the word pair and similarity mappings in your filtered test set in a dictionary called *filtered_gold_standard*. You may check the section, <i>"For your testing"</i>, below for the expected *filtered_gold_standard*.

(1 mark)

In [2]:
import nltk
from nltk.corpus import brown
from nltk.corpus import wordnet

nltk.download("brown")
nltk.download("wordnet")

# filtered_gold_standard stores the word pairs and their human-annotated similarity in your filtered test set
filtered_gold_standard = {}

###
# Your answer BEGINS HERE
###
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma

with open("set1.tab") as file:
    next(file)
    for line in file:
        strings = line.split("\t")
        filtered_gold_standard[(strings[0], strings[1])] = float(strings[2])
doc_freqs = {}
for para in brown.paras():
    processed_para_set = set()
    for sentence in para:
        for word in sentence:
            if word.isalpha():
                word = lemmatize(word.lower())
                processed_para_set.add(word)
    for word in processed_para_set:
        doc_freqs[word] = doc_freqs.get(word, 0) + 1
for pair in list(filtered_gold_standard):
    if doc_freqs.get(pair[0],0) < 8 or doc_freqs.get(pair[1],0) < 8:
        filtered_gold_standard.pop(pair, None)
###
# Your answer ENDS HERE
###

print(len(filtered_gold_standard))
print(filtered_gold_standard)


[nltk_data] Downloading package brown to /Users/ruilin/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/ruilin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
94
{('bread', 'butter'): 6.19, ('precedent', 'example'): 5.85, ('precedent', 'information'): 3.85, ('stock', 'live'): 3.73, ('tiger', 'zoo'): 5.87, ('tiger', 'organism'): 4.77, ('company', 'stock'): 7.08, ('money', 'dollar'): 8.42, ('noon', 'string'): 0.54, ('cup', 'liquid'): 5.9, ('drink', 'mouth'): 5.96, ('love', 'sex'): 6.77, ('book', 'paper'): 7.46, ('forest', 'graveyard'): 1.85, ('money', 'deposit'): 7.73, ('bird', 'cock'): 7.1, ('movie', 'theater'): 7.92, ('brother', 'monk'): 6.27, ('money', 'currency'): 9.04, ('baby', 'mother'): 7.85, ('cup', 'food'): 5.0, ('smart', 'student'): 4.62, ('cup', 'entity'): 2.15, ('money', 'possession'): 7.29, ('book', 'library'): 7.46, ('psychology', 'health'): 7.23, ('money', 'bank'): 8.5, ('tiger', 'animal'): 

<b>For your testing: </b>

In [3]:
assert(len(filtered_gold_standard) > 50 and len(filtered_gold_standard) < 100)

In [4]:
assert(filtered_gold_standard[('love', 'sex')] == 6.77)

<b>Instructions</b>: Here, you apply the second filtering. The second filtering is based on words with highly ambiguous senses and involves using the NLTK interface to WordNet. Here, you should remove any words which do not have a *single primary sense*. We define single primary sense here as either a) having only one sense (i.e. only one synset), or b) where the count (as provided by the WordNet `count()` method for the lemmas associated with a synset) of the most common sense is at least 4 times larger than the next most common sense. Note that a synset can be associated with multiple lemmas. You should only consider the count of your lemma. Also, you should remove any words where the primary sense is not a noun (this information is also in the synset). Store the synset corresponding to this primary sense in a dictionary for use in the next section. Given this definition, remove the word pairs from the test set where at least one of the words does not meet the above criteria.

When you have applied the two filtering steps, you should store all the word pair and similarity mappings in your filtered test set in a dictionary called *final_gold_standard*. You may check the section, <i>"For your testing"</i>, below for the expected *final_gold_standard*.

(1 mark)

In [5]:
# final_gold_standard stores the word pairs and their human-annotated similarity in your final filtered test set
final_gold_standard = {}

###
# Your answer BEGINS HERE
###
def get_sense_count(synset, word):
    for lemma in synset.lemmas():
        if lemma.name().lower() == word.lower():
            return lemma.count()

# Return the synset with the most common sense except synset.name() == exc
def get_most_common_sense_except(word, exc):
    most_common = None
    max_count = -1
    for synset in wordnet.synsets(word):
        if synset.name() == exc:
            continue
        count = get_sense_count(synset, word)
        if max_count < count:
            max_count = count
            most_common = synset
    return most_common

words = set()
for pair in filtered_gold_standard:
    words.add(pair[0])
    words.add(pair[1])

primary_sense = {}
for word in words:
    most_common = get_most_common_sense_except(word, "")
    if most_common.pos() != "n":
        continue
    second_common = get_most_common_sense_except(word, most_common.name())
    if second_common is None or get_sense_count(most_common, word) >= get_sense_count(second_common, word) * 4:
        primary_sense[word] = most_common

for pair in filtered_gold_standard:
    if pair[0] in primary_sense and pair[1] in primary_sense:
        final_gold_standard[pair] = filtered_gold_standard[pair]
###
# Your answer ENDS HERE
###

print(len(final_gold_standard))
print(final_gold_standard)

26
{('planet', 'moon'): 8.08, ('bread', 'butter'): 6.19, ('king', 'queen'): 8.58, ('journey', 'car'): 5.85, ('stock', 'egg'): 1.81, ('psychology', 'health'): 7.23, ('psychology', 'doctor'): 6.42, ('car', 'automobile'): 8.94, ('monk', 'slave'): 0.92, ('coast', 'forest'): 3.15, ('baby', 'mother'): 7.85, ('student', 'professor'): 6.81, ('professor', 'doctor'): 6.62, ('psychology', 'science'): 6.71, ('coast', 'shore'): 9.1, ('forest', 'graveyard'): 1.85, ('money', 'cash'): 9.08, ('bishop', 'rabbi'): 6.69, ('psychology', 'mind'): 7.69, ('football', 'basketball'): 6.81, ('brother', 'monk'): 6.27, ('coast', 'hill'): 4.38, ('alcohol', 'chemistry'): 5.54, ('journey', 'voyage'): 9.29, ('football', 'tennis'): 6.63, ('planet', 'galaxy'): 8.11}


<b>For your testing:</b>

In [6]:
assert(len(final_gold_standard) > 10 and len(final_gold_standard) < 40)

In [7]:
assert(final_gold_standard[('professor', 'doctor')] == 6.62)

## 2. Word similiarity scores with Lin similarity, NPMI and LSA (3 marks)

<b>Instructions</b>: Now you will create several dictionaries with similarity scores for pairs of words in your test set derived using the techniques discussed in class. The first of these is the Lin similarity for your word pairs using the information content of the Brown corpus, which you should calculate using the primary sense for each word derived above. You can use the built-in method included in the NLTK interface, you don't have to implement your own. 

When you're done, you should store the word pair and similarity mappings in a dictionary called *lin_similarities*. You may check the section, <i>"For your testing"</i>, below for the expected *lin_similarities*. 

(1 mark)

In [8]:
from nltk.corpus import wordnet_ic
nltk.download('wordnet_ic')

# lin_similarities stores the word pair and Lin similarity mappings
lin_similarities = {}

###
# Your answer BEGINS HERE
###
brown_ic = wordnet_ic.ic('ic-brown.dat')
for pair in final_gold_standard:
    lin_similarities[pair] = primary_sense[pair[0]].lin_similarity(primary_sense[pair[1]], brown_ic)
###
# Your answer ENDS HERE
###

print(lin_similarities)

[nltk_data] Downloading package wordnet_ic to
[nltk_data]     /Users/ruilin/nltk_data...
[nltk_data]   Package wordnet_ic is already up-to-date!
{('planet', 'moon'): 0.7808882364067532, ('bread', 'butter'): 0.711420490146294, ('king', 'queen'): 0.25872135992145145, ('journey', 'car'): -0.0, ('stock', 'egg'): -0.0, ('psychology', 'health'): 0.06004979886905243, ('psychology', 'doctor'): -0.0, ('car', 'automobile'): 1.0, ('monk', 'slave'): 0.2543108201944307, ('coast', 'forest'): -0.0, ('student', 'professor'): 0.26208607023317687, ('professor', 'doctor'): 0.7036526610448273, ('planet', 'galaxy'): -0.0, ('psychology', 'science'): 0.8474590505736942, ('football', 'tennis'): 0.7699955045932811, ('coast', 'shore'): 0.9632173804623256, ('forest', 'graveyard'): -0.0, ('money', 'cash'): 0.7888839126424345, ('psychology', 'mind'): 0.304017384194818, ('football', 'basketball'): 0.7536025025710653, ('brother', 'monk'): 0.24862817480738675, ('bishop', 'rabbi'): 0.6655650900427844, ('alcohol', 'che

<b>For your testing:</b>

In [9]:
assert(lin_similarities[('professor', 'doctor')] > 0.5 and lin_similarities[('professor', 'doctor')] < 1)

**Instructions:** Next, you will calculate Normalized PMI (NPMI) for your word pairs using word frequency derived from the Brown.

PMI is defined as:

\begin{equation*}
PMI = \log_2\left(\frac{p(x,y)}{p(x)p(y)}\right)
\end{equation*}

where

\begin{equation*}
p(x,y) = \frac{\text{Number of paragraphs with the co-occurrence of x and y}}{\sum_i \text{Number of word types in paragraph}_i}
\end{equation*}

\begin{equation*}
p(x) = \frac{\text{Number of paragraphs with the occurrence of x}}{\sum_i \text{Number of word types in paragraph}_i}
\end{equation*}

\begin{equation*}
p(y) = \frac{\text{Number of paragraphs with the occurrence of y}}{\sum_i \text{Number of word types in paragraph}_i}
\end{equation*}

with the sum over $i$ ranging over all paragraphs. Note that there are other ways PMI could be formulated.

NPMI is defined as:

\begin{equation*}
NPMI = \frac{PMI}{-\log_2(p(x,y))} = \frac{\log_2(p(x)p(y))}{\log_2(p(x,y))} - 1
\end{equation*}

Thus, when there is no co-occurrence, NPMI is -1. NPMI is normalized between [-1, +1].

You should use the same set up as you did to calculate document frequency above: paragraphs as documents, lemmatized, lower-cased, and with term frequency information removed by conversion to Python sets. You need to use the basic method for calculating PMI introduced in class (and also in the reading) which is appropriate for any possible definition of co-occurrence (here, there is co-occurrence when a word pair appears in the same paragraph), but you should only calculate PMI for the words in your test set. You must avoid building the entire co-occurrence matrix, instead you should keeping track of the sums you need for the probabilities as you go along. 

When you have calculated NPMI for all the pairs, you should store the word pair and NPMI-similarity mappings in a dictionary called *NPMI_similarities*. You may check the section, <i>"For your testing"</i>, below for the expected *NPMI_similarities*. 

(1 mark)

In [25]:
# NPMI_similarities stores the word pair and NPMI similarity mappings
NPMI_similarities = {}

###
# Your answer BEGINS HERE
###
import math

final_gold_standard_dict = set()
for pair in final_gold_standard:
    final_gold_standard_dict.add(pair[0])
    final_gold_standard_dict.add(pair[1])

posting_list = {}
para_id = 0
total_word_type_count = 0.0
for para in brown.paras():
    processed_para_set = set()
    for sentence in para:
        for word in sentence:
            if word.isalpha():
                word = lemmatize(word.lower())
                processed_para_set.add(word)
    total_word_type_count += len(processed_para_set)
    for word in processed_para_set:
        if word in final_gold_standard_dict:
            if word in posting_list:
                posting_list[word].add(para_id)
            else:
                posting_list[word] = {para_id}
    para_id += 1
    
def get_NPMI_for_brown(word1, word2):
    word1_para_count = len(posting_list[word1])
    word2_para_count = len(posting_list[word2])
    both_para_count = len(posting_list[word1].intersection(posting_list[word2]))
    if both_para_count == 0:
        return -1
    return (math.log((word1_para_count / total_word_type_count) * (word2_para_count / total_word_type_count), 2) / 
           math.log((both_para_count / total_word_type_count), 2) - 1)

for pair in final_gold_standard:
    NPMI_similarities[pair] = get_NPMI_for_brown(pair[0], pair[1])
###
# Your answer ENDS HERE
###

print(NPMI_similarities)

{('planet', 'moon'): 0.6580006162323315, ('bread', 'butter'): 0.6531272737497535, ('king', 'queen'): 0.4181407297139579, ('journey', 'car'): -1, ('stock', 'egg'): 0.3685128922213745, ('psychology', 'health'): -1, ('psychology', 'doctor'): 0.46132815837264096, ('car', 'automobile'): 0.5430334549802613, ('monk', 'slave'): -1, ('coast', 'forest'): 0.45787396913892864, ('student', 'professor'): 0.535495995400926, ('professor', 'doctor'): -1, ('planet', 'galaxy'): -1, ('psychology', 'science'): 0.5908740096534866, ('football', 'tennis'): -1, ('coast', 'shore'): 0.586151062984337, ('forest', 'graveyard'): -1, ('money', 'cash'): 0.44493834723517267, ('psychology', 'mind'): 0.44616660434351796, ('football', 'basketball'): 0.7161994042283006, ('brother', 'monk'): 0.4299318525693172, ('bishop', 'rabbi'): -1, ('alcohol', 'chemistry'): 0.6246376972254173, ('journey', 'voyage'): -1, ('baby', 'mother'): 0.5149353890388502, ('coast', 'hill'): 0.3393102883484622}


<b>For your testing:</b>

In [26]:
assert(NPMI_similarities[('professor', 'doctor')] == -1)

**Instructions:** As PMI matrix is very sparse and can be approximated well by a dense representation via singular value decomposition (SVD), you will derive similarity scores using the Latent Semantic Analysis (LSA) method, i.e. apply SVD and truncate to get a dense vector representation of a word type and then calculate cosine similarity between the two vectors for each word pair. You can use the Distributed Semantics notebook as a starting point, but note that since you are interested here in word semantics, you will be constructing a matrix where the (non-sparse) rows correspond to words in the vocabulary, and the (sparse) columns correspond to the texts where they appear (this is the opposite of the notebook). Again, use the Brown corpus, in the same format as with PMI and document frequency. After you have a matrix in the correct format, use `truncatedSVD` in `sklearn` to produce dense vectors of length k = 500, and then use cosine similarity to produce similarities for your word pairs. 

When you are done, you should store the word pair and LSA-similarity mappings in a dictionary called *LSA_similarities*. You may check the section, <i>"For your testing"</i>, below for the expected *LSA_similarities*. 

(1 mark)

In [36]:
# LSA_similarities stores the word pair and LSA similarity mappings
LSA_similarities = {}

###
# Your answer BEGINS HERE
###
vocabulary_brown_id = {}
voc_posting_lists = []
voc_id = 0
para_id = 0
for para in brown.paras():
    processed_para_set = set()
    for sentence in para:
        for word in sentence:
            if word.isalpha():
                word = lemmatize(word.lower())
                processed_para_set.add(word)
    for word in processed_para_set:
        if not word in vocabulary_brown_id:
            vocabulary_brown_id[word] = voc_id
            voc_id += 1
            voc_posting_lists.append({para_id : 1})
        else:
            voc_posting_lists[vocabulary_brown_id[word]][para_id] = 1
    para_id += 1

from sklearn.feature_extraction import DictVectorizer
vectorizer = DictVectorizer()
brown_matrix = vectorizer.fit_transform(voc_posting_lists)

from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=500)
brown_matrix_lowrank = svd.fit_transform(brown_matrix)

import numpy as np
for pair in final_gold_standard:
    word1_vec = brown_matrix_lowrank[vocabulary_brown_id[pair[0]]]
    word2_vec = brown_matrix_lowrank[vocabulary_brown_id[pair[1]]]
    LSA_similarities[pair] = np.dot(word1_vec, word2_vec) / np.sqrt(np.dot(word1_vec, word1_vec) * np.dot(word2_vec, word2_vec))
###
# Your answer ENDS HERE
###

print(LSA_similarities)

{('planet', 'moon'): 0.4071248849093906, ('bread', 'butter'): 0.3049675723687067, ('king', 'queen'): 0.10904775702191281, ('journey', 'car'): 0.01599791837633559, ('stock', 'egg'): 0.11575722372251904, ('psychology', 'health'): 0.010227228856426414, ('psychology', 'doctor'): 0.0999045326069421, ('car', 'automobile'): 0.35169964693598577, ('monk', 'slave'): -0.02045945193720879, ('coast', 'forest'): 0.1371097887924005, ('student', 'professor'): 0.2994524481075816, ('professor', 'doctor'): 0.09430660394096506, ('planet', 'galaxy'): 0.052180445927353304, ('psychology', 'science'): 0.2556933174676361, ('football', 'tennis'): 0.1280482493730215, ('coast', 'shore'): 0.4557127530150523, ('forest', 'graveyard'): 0.0586650629502895, ('money', 'cash'): 0.14956055863237422, ('psychology', 'mind'): 0.11342648558036686, ('football', 'basketball'): 0.23162123978928825, ('brother', 'monk'): 0.09452919746430721, ('bishop', 'rabbi'): 0.017262297398838618, ('alcohol', 'chemistry'): 0.11125685805483866, 

<b>For your testing:</b>

In [35]:
assert(LSA_similarities[('professor', 'doctor')] > 0 and LSA_similarities[('professor', 'doctor')] < 0.4)

## 3. Comparison with the Gold Standard (1 mark)


**Instructions:** Finally, you should compare all the similarities you've created to the gold standard you loaded and filtered in the first step. For this, you can use the Pearson correlation co-efficient (`pearsonr`), which is included in scipy (`scipy.stats`). Be careful converting your dictionaries to lists for this purpose, the data for the two datasets needs to be in the same order for correct comparison using correlation. Write a general function, then apply it to each of the similarity score dictionaries.

When you are done, you should put the result in a dictionary called *pearson_correlations* (use the keys: 'lin', 'NPMI', 'LSA'). You may check the section, <i>"For your testing"</i>, below for the expected *pearson_correlations*. 

<b>Hint:</b> All of the methods used here should be markedly above 0, but also far from 1 (perfect correlation); if you're not getting reasonable results, go back and check your code for bugs!  

(1 mark)


In [42]:
from scipy.stats import pearsonr

# pearson_correlations stores the pearson correlations with the gold standard of 'lin', 'NPMI', 'LSA'
pearson_correlations = {}

###
# Your answer BEGINS HERE
###
def dict_to_list(d):
    return [value for (key, value) in sorted(d.items())]

def pearson_correlation(x, y):
    return pearsonr(x, y)[0]

pearson_correlations['lin'] = pearson_correlation(dict_to_list(final_gold_standard), dict_to_list(lin_similarities))
pearson_correlations['NPMI'] = pearson_correlation(dict_to_list(final_gold_standard), dict_to_list(NPMI_similarities))
pearson_correlations['LSA'] = pearson_correlation(dict_to_list(final_gold_standard), dict_to_list(LSA_similarities))
###
# Your answer ENDS HERE
###

print(pearson_correlations)

{'NPMI': 0.18799888869410042, 'lin': 0.5301489978447533, 'LSA': 0.4115195991384225}


<b>For your testing:</b>

In [43]:
assert(pearson_correlations['lin'] > 0.4 and pearson_correlations['lin'] < 0.8)

## Challenge yourself: Improving the correlation (This part will NOT be marked)

You can try to derive a similarity score from word2vec vectors, using the Gensim interface, and compare it with the similarity scores you've created and the gold standard. Check the Gensim word2vec tutorial for details on the API: https://radimrehurek.com/gensim/models/word2vec.html. Again, you should use the Brown for this, but for word2vec you don't need to worry about paragraphs: feel free to train your model at the sentence level instead. Your vectors should have the same number of dimensions as LSA (500), and you need to run for 50 iterations. This may take a while (several minutes).

## A final word

Normally, we would not use a corpus as small as the Brown for the purposes of building distributional word vectors. Also, note that filtering our test set to just words we are likely to do well on would typically be considered cheating.